In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# To plot QQ plot
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

## Preprocess the data

# Prediction Generation

1. input data
2. Preprocess Data:
    - Replace -1 with 0s
2. Do something about NA - Create submodels to impute missing values (Random Forest)
2. Fit the model
3. Generate Predictions
4. Generate Confidence Score


In [7]:
prediction_data_1 = pd.read_csv('v2_Inputs_for_Full_Predictions_2_cleaned.csv')

In [8]:
prediction_data_1.shape

(2489558, 31)

In [9]:
prediction_data_1.isna().sum()

Var_1                   0
Var_2                   0
Var_3                   0
Var_4                   0
Var_5_Class_1           0
Var_5_Class_2           0
Var_5_Class_3           0
Var_5_Class_4           0
Var_5_Class_5           0
Var_5_Class_6           0
Var_5_Class_7           0
Var_5_Class_8           0
Var_6_Class_1           0
Var_6_Class_2           0
Var_6_Class_3           0
Var_6_Class_4           0
Var_6_Class_5           0
Var_6_Class_6           0
Var_6_Class_7           0
Var_6_Class_8           0
Var_6_Class_9           0
Var_6_Class_10          0
Var_6_Class_11          0
Var_6_Class_12          0
Var_7                   0
Var_8                   0
Var_9                   0
Var_10                  0
Var_11            1765433
Var_12            1502976
Var_13                  0
dtype: int64

In [10]:
data_numeric = ['Var_1', 'Var_2', 'Var_3', 'Var_4', 'Var_7', 'Var_8', 'Var_11', 'Var_12', 'Var_13']

In [11]:
# Function to idenfity skewness
def data_skewness(df):
    for i in data_numeric:
        print(i, ":  ", df[i].skew())

In [12]:
data_skewness(prediction_data_1)

Var_1 :   3.9019749056127604
Var_2 :   1.2858898594763082
Var_3 :   18.706620448485587
Var_4 :   -1.2845748252297855
Var_7 :   39.22619102899415
Var_8 :   9.884109368629714
Var_11 :   0.15954913972383067
Var_12 :   -0.11080973977119667
Var_13 :   -0.13667133273249232


In [13]:
# Removing skewness for positivly skewed data
prediction_data_1["Var_1"] = np.sqrt(prediction_data_1["Var_1"])
prediction_data_1["Var_2"] = np.sqrt(prediction_data_1["Var_2"])
prediction_data_1["Var_3"] = np.cbrt(prediction_data_1["Var_3"])
prediction_data_1["Var_7"] = np.cbrt(prediction_data_1["Var_7"])
prediction_data_1["Var_8"] = np.cbrt(prediction_data_1["Var_8"])

# Removing skewness for negatively skewed data
#data["Var_12"] = np.power(data["Var_12"], 2)
#prediction_data_1["Var_4"] = np.power(prediction_data_1["Var_4"], 6)

In [14]:
data_skewness(prediction_data_1)

Var_1 :   0.49872883873460444
Var_2 :   0.3576557463036899
Var_3 :   0.9441237516931893
Var_4 :   -1.2845748252297855
Var_7 :   3.942982610036619
Var_8 :   0.3741561309053679
Var_11 :   0.15954913972383067
Var_12 :   -0.11080973977119667
Var_13 :   -0.13667133273249232


## Imputing missing data - Var 11
- Create model to predict data.

In [15]:
len(prediction_data_1[prediction_data_1.Var_11.isna() & prediction_data_1.Var_12.isna()]), len(prediction_data_1[prediction_data_1.Var_11.isna()]), len(prediction_data_1[prediction_data_1.Var_12.isna()])

(1502976, 1765433, 1502976)

We notied that whenever Var_12 is missing, Var_11 is also missing. We will 1st build a model to predict Var 11 and hold out Var 12. Then use final model to predict Var_12

In [16]:
#cols = prediction_data_1.columns
cols = ['Var_1', 'Var_2', 'Var_3', 'Var_4', 'Var_5_Class_1', 'Var_5_Class_2',
       'Var_5_Class_3', 'Var_5_Class_4', 'Var_5_Class_5', 'Var_5_Class_6',
       'Var_5_Class_7', 'Var_5_Class_8', 'Var_6_Class_1', 'Var_6_Class_2',
       'Var_6_Class_3', 'Var_6_Class_4', 'Var_6_Class_5', 'Var_6_Class_6',
       'Var_6_Class_7', 'Var_6_Class_8', 'Var_6_Class_9', 'Var_6_Class_10',
       'Var_6_Class_11', 'Var_6_Class_12', 'Var_7', 'Var_8', 'Var_9', 'Var_10','Var_11','Var_13']

In [17]:
impute_independent_var = prediction_data_1[cols]

In [18]:
impute_independent_var.drop(impute_independent_var[pd.isna(impute_independent_var.Var_11)].index, inplace=True)

In [19]:
# Segregating data in independent and dependent variables

X_impute = impute_independent_var.drop(['Var_11'], axis=1)
y_impute = impute_independent_var['Var_11']

In [20]:
from sklearn.model_selection import train_test_split

train_X_impute, test_X_impute, train_y_impute, test_y_impute = train_test_split(X_impute, y_impute, random_state=122, stratify = y_impute)

In [21]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean

In [22]:
from sklearn.ensemble import RandomForestRegressor

var_11_model = RandomForestRegressor(n_estimators=10, random_state=12)

In [23]:
# # define evaluation procedure
# cv_impute = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate model
# scores_impute = cross_val_score(var_11_model, X_impute, y_impute, scoring='accuracy', cv=cv_impute, n_jobs=-1)
# # summarize performance
# print('Accuracy %.3f' % mean(scores_impute))

In [24]:
var_11_model.fit(train_X_impute, train_y_impute)

RandomForestRegressor(n_estimators=10, random_state=12)

In [25]:
impute_independent_var = pd.read_csv('v2_Inputs_for_Full_Predictions_3_cleaned.csv')

In [26]:
X_impute = impute_independent_var.drop(['Var_11', 'Var_12'], axis=1)
y_impute = impute_independent_var['Var_11']

In [34]:
impute_independent_var['Var_11'] = var_11_model.predict(X_impute)

In [35]:
impute_independent_var[impute_independent_var.Var_11.isnull()]

,Var_1,Var_2,Var_3,Var_4,Var_5_Class_1,Var_5_Class_2,Var_5_Class_3,Var_5_Class_4,Var_5_Class_5,Var_5_Class_6,...,Var_6_Class_12,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13,Val_11_predicte,Val_11


In [36]:
import math

In [37]:
impute_independent_var.Var_11 = impute_independent_var.Var_11.apply(lambda x: format(x, '.2f'))

In [41]:
impute_independent_var.isna().sum()

Var_1                   0
Var_2                   0
Var_3                   0
Var_4                   0
Var_5_Class_1           0
Var_5_Class_2           0
Var_5_Class_3           0
Var_5_Class_4           0
Var_5_Class_5           0
Var_5_Class_6           0
Var_5_Class_7           0
Var_5_Class_8           0
Var_6_Class_1           0
Var_6_Class_2           0
Var_6_Class_3           0
Var_6_Class_4           0
Var_6_Class_5           0
Var_6_Class_6           0
Var_6_Class_7           0
Var_6_Class_8           0
Var_6_Class_9           0
Var_6_Class_10          0
Var_6_Class_11          0
Var_6_Class_12          0
Var_7                   0
Var_8                   0
Var_9                   0
Var_10                  0
Var_11                  0
Var_12            2412957
Var_13                  0
dtype: int64

In [40]:
impute_independent_var.drop("Val_11_predicte", inplace=True, axis=1)

In [225]:
impute_independent_var.shape, prediction_data_1.shape

((2489558, 31), (2489558, 31))

In [42]:
impute_independent_var.to_csv("Var_11_imputed.csv", index=False)

## Predict Val_12

In [50]:
impute_independent_var = pd.read_csv("Var_11_imputed_2.csv")

In [51]:
impute_independent_var.shape

(2489558, 31)

In [54]:
data_skewness(impute_independent_var)

Var_1 :   0.49872883873460444
Var_2 :   0.3576557463036899
Var_3 :   0.9441237516931893
Var_4 :   -1.2845748252297855
Var_7 :   3.942982610036619
Var_8 :   0.3741561309053679
Var_11 :   2.875985513437193
Var_12 :   -0.11080973977119667
Var_13 :   -0.13667133273249232


In [53]:
# Removing skewness. Taking root for positive skewed data and power for negatively skewed data
impute_independent_var["Var_1"] = np.sqrt(impute_independent_var["Var_1"])
impute_independent_var["Var_2"] = np.sqrt(impute_independent_var["Var_2"])
impute_independent_var["Var_3"] = np.cbrt(impute_independent_var["Var_3"])
impute_independent_var["Var_7"] = np.cbrt(impute_independent_var["Var_7"])
impute_independent_var["Var_8"] = np.cbrt(impute_independent_var["Var_8"])
impute_independent_var["Var_11"] = np.power(impute_independent_var["Var_11"], 5)
#prediction_data_1["Var_4"] = np.power(prediction_data_1["Var_4"], 6)

In [55]:
impute_independent_var.isna().sum()

Var_1                   0
Var_2                   0
Var_3                   0
Var_4                   0
Var_5_Class_1           0
Var_5_Class_2           0
Var_5_Class_3           0
Var_5_Class_4           0
Var_5_Class_5           0
Var_5_Class_6           0
Var_5_Class_7           0
Var_5_Class_8           0
Var_6_Class_1           0
Var_6_Class_2           0
Var_6_Class_3           0
Var_6_Class_4           0
Var_6_Class_5           0
Var_6_Class_6           0
Var_6_Class_7           0
Var_6_Class_8           0
Var_6_Class_9           0
Var_6_Class_10          0
Var_6_Class_11          0
Var_6_Class_12          0
Var_7                   0
Var_8                   0
Var_9                   0
Var_10                  0
Var_11                  0
Var_12            1502976
Var_13                  0
dtype: int64

In [56]:
impute_independent_var.drop(impute_independent_var[pd.isna(impute_independent_var.Var_12)].index, inplace=True)

In [57]:
# Segregating data in independent and dependent variables

X_impute = impute_independent_var.drop(['Var_12', 'Var_11'], axis=1)
y_impute = impute_independent_var['Var_12']

In [58]:
from sklearn.model_selection import train_test_split

train_X_impute, test_X_impute, train_y_impute, test_y_impute = train_test_split(X_impute, y_impute, random_state=122, stratify = y_impute)

In [59]:
from sklearn.ensemble import RandomForestRegressor

var_12_model = RandomForestRegressor(n_estimators=10, random_state=12)

In [60]:
train_X_impute.isna().sum()

Var_1             0
Var_2             0
Var_3             0
Var_4             0
Var_5_Class_1     0
Var_5_Class_2     0
Var_5_Class_3     0
Var_5_Class_4     0
Var_5_Class_5     0
Var_5_Class_6     0
Var_5_Class_7     0
Var_5_Class_8     0
Var_6_Class_1     0
Var_6_Class_2     0
Var_6_Class_3     0
Var_6_Class_4     0
Var_6_Class_5     0
Var_6_Class_6     0
Var_6_Class_7     0
Var_6_Class_8     0
Var_6_Class_9     0
Var_6_Class_10    0
Var_6_Class_11    0
Var_6_Class_12    0
Var_7             0
Var_8             0
Var_9             0
Var_10            0
Var_13            0
dtype: int64

In [61]:
var_12_model.fit(train_X_impute, train_y_impute)

RandomForestRegressor(n_estimators=10, random_state=12)

In [62]:
impute_independent_var = pd.read_csv('Var_11_imputed.csv')

X_impute = impute_independent_var.drop(['Var_11', 'Var_12'], axis=1)
y_impute = impute_independent_var['Var_12']

In [63]:
impute_independent_var['Var_12'] = var_12_model.predict(X_impute)

In [64]:
impute_independent_var[impute_independent_var.Var_12.isnull()]

,Var_1,Var_2,Var_3,Var_4,Var_5_Class_1,Var_5_Class_2,Var_5_Class_3,Var_5_Class_4,Var_5_Class_5,Var_5_Class_6,...,Var_6_Class_10,Var_6_Class_11,Var_6_Class_12,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13


In [65]:
impute_independent_var.Var_12 = impute_independent_var.Var_12.apply(lambda x: format(x, '.2f'))

In [66]:
impute_independent_var.Var_12.isna().sum()

0

In [243]:
#impute_independent_var.Var_12.fillna(impute_independent_var.Val_12_predicte, inplace=True)

In [ ]:
#impute_independent_var[impute_independent_var.Var_12.isna()]['Var_12'] = impute_independent_var[impute_independent_var.Var_12.isna()]['Var_12'].replace(impute_independent_var.Var_12, impute_independent_var.Val_12_predicte) 

In [ ]:
# for i in impute_independent_var.Var_12:
#     if(math.isnan(i)):
#         impute_independent_var.Var_12 = impute_independent_var.Val_12_predicte

In [244]:
#impute_independent_var.drop("Val_12_predicte", inplace=True, axis=1)

In [67]:
impute_independent_var.isna().sum()

Var_1             0
Var_2             0
Var_3             0
Var_4             0
Var_5_Class_1     0
Var_5_Class_2     0
Var_5_Class_3     0
Var_5_Class_4     0
Var_5_Class_5     0
Var_5_Class_6     0
Var_5_Class_7     0
Var_5_Class_8     0
Var_6_Class_1     0
Var_6_Class_2     0
Var_6_Class_3     0
Var_6_Class_4     0
Var_6_Class_5     0
Var_6_Class_6     0
Var_6_Class_7     0
Var_6_Class_8     0
Var_6_Class_9     0
Var_6_Class_10    0
Var_6_Class_11    0
Var_6_Class_12    0
Var_7             0
Var_8             0
Var_9             0
Var_10            0
Var_11            0
Var_12            0
Var_13            0
dtype: int64

In [68]:
impute_independent_var.to_csv("v2_Inputs_for_Full_Predictions_3_imputed.csv", index=False)